# Deep Learning for NLP WS17/18
## Exercise Sheet 3 - Pytorch Introduction
This exercise sheet is due on 21.11.17 11:59 pm. There is a total of 8
points for this exercise sheet. Please send your solution in a
suitable format to [beroth@cis.uni-muenchen.de](mailto:beroth@cis.uni-muenchen.de). Please submit a
completed version of this file in Python 3. You may submit in teams of
2 or 3 students.

Please rename the file to pytorch_intro_last_names.ipynb

### Exercise 1
#### Derive the gradient of the negative log likelihood for logistic regression (3 points)

Negative log likelihood for logistic regression:

$$ NLL(\vec \theta) =  - \sum_{i=1}^m y^{(i)} \log \sigma(\vec{\theta}^T \vec{x^{(i)}}) + (1-y^{(i)}) \log (1 - \sigma(\vec{\theta}^T \vec{x^{(i)}}))$$

Derive the expression for

$$ \nabla_\theta NLL(\vec \theta) $$

For each step give the exact rule you used.

Note that you can use latex math syntax in notebooks.

#### TODO

### Exercise 2
#### Logistic regression on the Boston Housing Dataset (5 points)

Usefull Pytorch tutorials and resources are mentioned in the lecture slides. 


The Boston Housing Dataset is often used as a logistic regression example. It has 14 features and 2 prototasks. See <https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html> for details. The dataset is provided with the sklearn module.

You will have to complete the code where marked with ***TODO***

First, we import Pytorch, the Boston dataset, math and shuffle. For installation of Pytorch check <http://pytorch.org/>.

In [1]:
import torch
import torch.nn as nn
import sklearn.datasets
from random import shuffle
import torch.optim as optim
import numpy as np
import math
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data


We define a method which returns a boolean array that helps providing a shuffled train, val, test dataset split

In [2]:
def random_train_dev_test_split(num_total_items, train_ratio = 0.5, dev_ratio = 0.25):
    num_train_items = math.floor(num_total_items * train_ratio)
    num_dev_items = math.floor(num_total_items * dev_ratio)
    num_test_items = num_total_items - num_train_items - num_dev_items
    split = [0] * num_train_items + [1] * num_dev_items + [2] * num_test_items
    shuffle(split)
    split = np.asarray(split)
    return split == 0, split == 1, split == 2


Now we start the main program. At first we get the dataset from the sklearn module and assign features and labels to x and y.

In [3]:
boston = sklearn.datasets.load_boston()
x = boston.data
y = boston.target
num_items, num_features = x.shape



#### TODO: Get a 50%-25%-25% split of the shuffled(!) training data (you can use the provided method random_train_dev_test_split)

In [4]:
train_spl, dev_spl, test_spl = random_train_dev_test_split(num_items)

#### TODO: get a k x n -dimensional matrix, where k is the number of items in the training data, n the number of features.

In [5]:
train_x = x[train_spl] 

#### TODO: get a k x 1 dimensional matrix of the training targets. Hint: use np.expand_dims(...) to get an extra dimension (i.e. a matrix instead of an vector).


In [6]:
train_y = np.expand_dims(y[train_spl], 1) 

#### TODO: get the dev feature matrix with associated dev targets

In [7]:
dev_x = x[dev_spl]
dev_y = np.expand_dims(y[dev_spl], 1)

#### TODO: implement a shuffle method for train_x and train_y so that after shuffling each data row is still correctly matched with its label.

In [8]:
def unison_shuffled(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

We create the Linear Regression class that inherits from the Pytorch nn module. Information about __init__ and __forward__ can be found in the lecture slides.

In [13]:
class LinearRegression(nn.Module):
    def __init__(self, num_features, hidden_size):
        super(LinearRegression, self).__init__()
        self.final_layer = nn.Linear(num_features, 1)
    def forward(self, x):
        return self.final_layer(x)
linreg_model = LinearRegression(num_features, 10)

optimizer = optim.SGD(linreg_model.parameters(), lr=0.00001)


#### TODO: replace by using the Adam optimizer with learning rate 0.001.

In [14]:
optimizer = torch.optim.Adam(linreg_model.parameters(), lr=0.001)

Definition of Mean Squared Error as loss criterion. The __np_to_var__ method creates a torch float variable and exists purely for readability.

In [15]:
criterion = nn.MSELoss()
def np_to_var(np_array):
    return Variable(torch.from_numpy(np_array).float())

The training iterates over the data 100 times. The gradient is backpropagated after every training example and total loss is printed at the end of each epoch.

#### TODO: What effect is caused by the unison_shuffle method?

 #### TODO: print loss on development data at the end of each epoch

In [16]:

num_epochs = 100
for epoch in range(num_epochs):
    loss_accum = 0.0
    train_x, train_y = unison_shuffled(train_x, train_y)
    for i in range(len(train_y)):
        x_i = np_to_var(train_x[i])
        y_i = np_to_var(train_y[i])
        optimizer.zero_grad()   # zero the gradient buffers
        output = linreg_model.forward(x_i)
        loss = criterion(output, y_i)
        loss_accum += loss.data[0]
        loss.backward()
        optimizer.step()    # Does the update
    print("train loss:", loss_accum/len(train_y))
   
    output_dev = linreg_model.forward(np_to_var(dev_x)) # TODO
    loss = criterion(output_dev, np_to_var(dev_y))# TODO
    print("dev loss:", loss.data[0])# TODO

train loss: 208.09283326368865
dev loss: 83.00550842285156
train loss: 106.52346815336907
dev loss: 67.3156509399414
train loss: 94.25656272849146
dev loss: 63.517215728759766
train loss: 87.2278339436587
dev loss: 66.9278793334961
train loss: 83.73745275678006
dev loss: 60.528358459472656
train loss: 79.51834368711478
dev loss: 59.027015686035156
train loss: 74.8202300498704
dev loss: 59.75569534301758
train loss: 73.54105619417932
dev loss: 55.34064483642578
train loss: 75.97731993317359
dev loss: 54.172149658203125
train loss: 70.29437990599632
dev loss: 53.33812713623047
train loss: 67.64432153252888
dev loss: 53.999759674072266
train loss: 63.9170387622407
dev loss: 59.18790817260742
train loss: 62.09293230215743
dev loss: 61.243412017822266
train loss: 63.90708964515808
dev loss: 52.37233352661133


KeyboardInterrupt: 

#### Optional: Implement the above with batch size > 1